In [1]:
import sys
sys.path.append('../')
import numpy as np
from GroupingAlgorithm_v3 import *
from HEEM_VQE_Functions import *
from utils import Label2Chain, get_backend_conectivity
from qiskit import IBMQ
from qiskit_nature.drivers import PyQuanteDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit.opflow import converters
from qiskit.opflow.primitive_ops import Z2Symmetries
from time import time
import networkx as nx

Obtaing the conectivity of a real quantum hardware.

In [2]:
IBMQ.load_account()
provider    = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') # put here your provider
backend_exp = provider.get_backend('ibmq_toronto') # select a quantum computer
WC_exp      = get_backend_conectivity(backend_exp)

In [3]:
G_exp=nx.Graph()
G_exp.add_nodes_from(range(27))
G_exp.add_edges_from(WC_exp)

Groups for different molecules. The grouping strategies considered are: non grouping, TPB, Bell+TPB, 2Q+TPB and HEEM.

In [4]:
t1 = time()

molecules = ['H .0 .0 .0; H .0 .0 0.761'  #0.88
             ,
             'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'
             ,
            'H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339'
             ,
            'O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0'
             ,
             'H 0.0 0.0 0.0; F 0.0 0.0 0.995'
             ,
            'Cl 0.0 0.0 0.0; H 0.0 0.0 1.0' 
            ]


for molecule in molecules :
    
    print( 'molecule: '+molecule )

    driver = PyQuanteDriver(atoms=molecule)
    qmolecule = driver.run()
    problem = ElectronicStructureProblem(driver)

    # Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

    # Hamiltonian
    main_op = second_q_ops[0]

    # Setup the mapper and qubit converter
    mapper = JordanWignerMapper()

    converter = QubitConverter(mapper=mapper)

    # The fermionic operators are mapped to qubit operators
    num_particles = (problem.grouped_property_transformed.get_property("ParticleNumber").num_alpha,
                     problem.grouped_property_transformed.get_property("ParticleNumber").num_beta)
    qubit_op   = converter.convert(main_op, num_particles=num_particles)
    
    num_qubits = qubit_op.num_qubits
    WC = list(range(num_qubits))
    WC = list(permutations(list(range(num_qubits)),2))
    G=nx.Graph()
    G.add_nodes_from(range(num_qubits))
    G.add_edges_from(WC)
    
    print('number of qubits:', num_qubits)

    paulis, coeff, labels = Label2Chain(qubit_op)
    print( 'number of Paulis:', len(coeff) )
    
    Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
    print( 'number of TPB groups:', len(Groups_tpb) )
    
    Groups_bell, Measurements_bell, T = groupingWithOrder( paulis, G )
    print('number of EM groups:', len(Groups_bell) )

    Groups_HEEM, Measurements_HEEM,T = groupingWithOrder( paulis, G_exp )
    print('number of HEEM groups :', len(Measurements_HEEM) )
    
    print('----------------------------------------')
    
t2 = time()
print(t2-t1)

molecule: H .0 .0 .0; H .0 .0 0.761
number of qubits: 4
number of Paulis: 15
number of TPB groups: 5
number of EM groups: 2
number of HEEM groups : 2
----------------------------------------
molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619


C:\Users\lucia\AppData\Local\Temp/ipykernel_18488/1115234347.py:21: DeprecationWarning: The PyQuanteDriver class is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the PyQuanteDriver class from qiskit_nature.drivers.second_quantization.pyquanted.
  driver = PyQuanteDriver(atoms=molecule)
C:\Users\lucia\miniconda3\lib\site-packages\qiskit_nature\drivers\pyquanted\pyquantedriver.py:93: DeprecationWarning: The HFMethodType enum is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the HFMethodType enum from qiskit_nature.drivers.second_quantization.
  hf_method = HFMethodType.RHF
C:\Users\lucia\miniconda3\lib\site-packages\qiskit_nature\drivers\pyquanted\pyquantedriver.py:99: DeprecationWarning: The BaseDriver class is deprecated as of version 0.2.0 and will be removed no sooner than 3 months after the release. Instead use the BaseDriver class from qiskit_nature.drivers.second_

number of qubits: 12
number of Paulis: 631
number of TPB groups: 136
number of EM groups: 43
number of HEEM groups : 105
----------------------------------------
molecule: H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339
number of qubits: 14
number of Paulis: 666
number of TPB groups: 140
number of EM groups: 46
number of HEEM groups : 106
----------------------------------------
molecule: O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0
number of qubits: 14
number of Paulis: 1086
number of TPB groups: 224
number of EM groups: 59
number of HEEM groups : 183
----------------------------------------
molecule: H 0.0 0.0 0.0; F 0.0 0.0 0.995
number of qubits: 12
number of Paulis: 631
number of TPB groups: 136
number of EM groups: 43
number of HEEM groups : 105
----------------------------------------
molecule: Cl 0.0 0.0 0.0; H 0.0 0.0 1.0
number of qubits: 20
number of Paulis: 5851
number of TPB groups: 1320
number of EM groups: 214
number of HEEM groups : 867
-----------------------

In [5]:
# v1
# molecule: H .0 .0 .0; H .0 .0 0.761
# number of qubits: 4
# number of Paulis: 15
# number of TPB groups: 5
# number of Bell groups: 2
# number of 2Q groups v1: 2
# number of 2Q groups v2: 2
# number of HEEM groups : 2
# ----------------------------------------
# molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619
# number of qubits: 12
# number of Paulis: 631
# number of TPB groups: 136
# number of Bell groups: 39
# number of 2Q groups v1: 39
# number of 2Q groups v2: 39
# number of HEEM groups : 101
# ----------------------------------------
# molecule: H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339
# number of qubits: 14
# number of Paulis: 666
# number of TPB groups: 140
# number of Bell groups: 41
# number of 2Q groups v1: 41
# number of 2Q groups v2: 41
# number of HEEM groups : 126
# ----------------------------------------
# molecule: O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0
# number of qubits: 14
# number of Paulis: 1086
# number of TPB groups: 224
# number of Bell groups: 58
# number of 2Q groups v1: 58
# number of 2Q groups v2: 58
# number of HEEM groups : 174
# ----------------------------------------
# 349.39394330978394

# v2
# molecule: H .0 .0 .0; H .0 .0 0.761
# number of qubits: 4
# number of Paulis: 15
# number of TPB groups: 5
# number of Bell groups: 2
# number of 2Q groups v1: 2
# number of 2Q groups v2: 2
# number of HEEM groups : 2
# ----------------------------------------
# molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619
# number of qubits: 12
# number of Paulis: 631
# number of TPB groups: 136
# number of Bell groups: 39
# number of 2Q groups v1: 39
# number of 2Q groups v2: 39
# number of HEEM groups : 101
# ----------------------------------------
# molecule: H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339
# number of qubits: 14
# number of Paulis: 666
# number of TPB groups: 140
# number of Bell groups: 41
# number of 2Q groups v1: 41
# number of 2Q groups v2: 41
# number of HEEM groups : 126
# ----------------------------------------
# molecule: O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0
# number of qubits: 14
# number of Paulis: 1086
# number of TPB groups: 224
# number of Bell groups: 58
# number of 2Q groups v1: 58
# number of 2Q groups v2: 58
# number of HEEM groups : 174
# ----------------------------------------
# 248.50871014595032

In [6]:
import qiskit.tools.jupyter
%qiskit_version_table

C:\Users\lucia\miniconda3\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
